In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from cv2 import aruco
import math
import shutil
from scipy.spatial.transform import Rotation as R
from pyquaternion import Quaternion
import math
import json
from tqdm import tqdm
import os.path as osp
import traceback

from mmengine.structures import InstanceData
from mmdet3d.structures import Det3DDataSample, CameraInstance3DBoxes

from devkit.computation.transporter import Transporter
from devkit.computation import *
from devkit.configs import aruco_size_by_id, aruco_2_center_mat, aruco_ids_to_detect  

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
src_dirs = []

root_dir = '/home/alan_khang/Desktop/adam_dataset'
transporter_name = ['t1', 't2']
date_dir = ['2025_07_25', '2025_07_29', '2025_07_30', '2025_07_31', '2025_08_01', '2025_08_02', '2025_08_03', '2025_08_04']

for transporter in transporter_name:
    for date in date_dir:
        dataset_root = osp.join(root_dir, f'{transporter}_{date}')
        if not osp.exists(dataset_root):
            print(f"Dataset root {dataset_root} does not exist.")
            continue
        scenes_by_date = sorted(os.listdir(dataset_root))
        scenes_by_date = list(filter(lambda x: date in x, scenes_by_date))
        scenes_by_date = [osp.join(dataset_root, s) for s in scenes_by_date]
        scenes_by_date = [s for s in scenes_by_date if osp.isdir(s)]
        src_dirs.extend(scenes_by_date)

Dataset root /home/alan_khang/Desktop/adam_dataset/t2_2025_07_29 does not exist.
Dataset root /home/alan_khang/Desktop/adam_dataset/t2_2025_07_30 does not exist.
Dataset root /home/alan_khang/Desktop/adam_dataset/t2_2025_07_31 does not exist.
Dataset root /home/alan_khang/Desktop/adam_dataset/t2_2025_08_01 does not exist.
Dataset root /home/alan_khang/Desktop/adam_dataset/t2_2025_08_02 does not exist.
Dataset root /home/alan_khang/Desktop/adam_dataset/t2_2025_08_03 does not exist.
Dataset root /home/alan_khang/Desktop/adam_dataset/t2_2025_08_04 does not exist.


In [3]:
offset_px = 5

correct_rot_mat = np.eye(3)

overlay_color = [0, 0, 0]

guilder_width = 0.58
guilder_height = 0.304
guilder_length = 0.488

distance_between_ground_and_bottom_aruco = 0.094
distance_between_guilder_rear_and_bottom_aruco = 0.04

guilder_whl = np.array([guilder_width, guilder_height, guilder_length])

guilder_center_coord_in_aruco_coord = np.array([
    [1., 0., 0., 0.],
    [0., 1., 0., (guilder_height / 2) - distance_between_ground_and_bottom_aruco],  
    [0., 0., 1., -((guilder_length / 2) - distance_between_guilder_rear_and_bottom_aruco)],   
    [0., 0., 0., 1.]], dtype=np.float32)

annotated_img_dir_name = 'processed/annotated_images'
annotated_dir_name = 'processed/annotations'
img_dir_name = 'processed/images'

transporter = Transporter()

for src_dir in src_dirs:
    raw_color_dir = os.path.join(src_dir, 'color')
    annotated_img_dir = os.path.join(src_dir, annotated_img_dir_name)
    annotated_dir = os.path.join(src_dir, annotated_dir_name)
    img_dir = os.path.join(src_dir, img_dir_name) 

    if os.path.exists(annotated_img_dir):
        shutil.rmtree(annotated_img_dir)

    if os.path.exists(annotated_dir):
        shutil.rmtree(annotated_dir)

    if os.path.exists(img_dir):
        shutil.rmtree(img_dir)

    os.makedirs(annotated_img_dir, exist_ok=True)
    os.makedirs(annotated_dir, exist_ok=True)
    os.makedirs(img_dir, exist_ok=True)

    img_paths = sorted(glob.glob(os.path.join(raw_color_dir, '*.jpg')))
    for img_path in tqdm(img_paths):
        try: 
            json_path = img_path.replace('color', 'info').replace('.jpg', '.json')
            if not os.path.exists(json_path):
                print(f"JSON file {json_path} does not exist. Skipping.")
                continue
            with open(json_path, 'r') as f:
                img_info = json.load(f)
            img_filename = os.path.basename(img_path)
            intr = np.array(img_info['camera_intrinsic'], dtype=np.float32)
            color_wh = (img_info['image_size']['width'], img_info['image_size']['height'])
            expected_size = (img_info['depth_size']['width'], img_info['depth_size']['height'])
            img = cv2.imread(img_path)

            if img.shape[:2][::-1] != color_wh:
                raise ValueError(
                    f"Image {img_filename} has unexpected size {img.shape[:2][::-1]}, expected {color_wh}.")

            img, intr = resizeWithCropFactor(img, expected_size, intrinsic=np.array(intr, dtype=np.float32))
            resized_img = img.copy()

            (aruco_boxes, aruco_centers, 
            aruco_detected_corners, aruco_detected_ids) = transporter.detect_aruco_markers_by_id(
                resized_img, 
                aruco_ids_to_detect,
                BGR_format=True)
            num_markers = len(aruco_centers)

            if num_markers == 0:
                #print(f"No ArUco markers detected in {img_filename}. Skipping.")
                continue

            markers_sizes = [aruco_size_by_id[id] for id in aruco_detected_ids]
            processed_img = np.ascontiguousarray(img.copy())
            rvecs, tvecs = transporter.estimate_arucos_poses(
                processed_img, 
                intr, 
                np.zeros(8, dtype=np.float64),
                markers_sizes, 
                aruco_detected_corners)

            assert len(rvecs) == len(aruco_detected_ids)

            cam_2_global_poses_in_frame = []
            aruco_detected = dict()

            for rvec, tvec, aruco_id, aruco_corners in zip(rvecs, tvecs, aruco_detected_ids, aruco_detected_corners):
                if rvec is None or tvec is None:
                    print(f"Skipping marker {aruco_id} due to None rvec or tvec.")
                    continue
                rmat, _ = cv2.Rodrigues(rvec)
                cam_2_aruco_quat = R.from_matrix(rmat).as_quat(scalar_first=True)
                cam_2_aruco_pose = np.eye(4, dtype=np.float32)
                cam_2_aruco_pose[:3, :3] = rmat
                cam_2_aruco_pose[:3, 3] = tvec.flatten()

                aruco_pose_2_global_pose = aruco_2_center_mat[aruco_id]
                cam_2_global_pose = cam_2_aruco_pose @ aruco_pose_2_global_pose

                cam_2_global_poses_in_frame.append(cam_2_global_pose)

                aruco_box = aruco_corners[0].astype(np.int16).squeeze()
                box_xyxy = [int(np.min(aruco_box[:, 0])), int(np.min(aruco_box[:, 1])),
                            int(np.max(aruco_box[:, 0])), int(np.max(aruco_box[:, 1]))]
                aruco_detected[str(aruco_id)] = {
                    'rot': cam_2_aruco_quat.tolist(),
                    'trans': tvec.flatten().tolist(),
                    'box_2d_xyxy': box_xyxy,
                    'aruco_2_global_rot': aruco_pose_2_global_pose.tolist()}

            assert len(cam_2_global_poses_in_frame) > 0
            translations = np.array([T[:3, 3] for T in cam_2_global_poses_in_frame])
            avg_translation = np.mean(translations, axis=0)

            # Average rotation using quaternions
            rotations = R.from_matrix([T[:3, :3] for T in cam_2_global_poses_in_frame])
            avg_rotation = R.mean(rotations).as_matrix()
        
            cam_2_global_pose_avg = np.eye(4)
            cam_2_global_pose_avg[:3, :3] = avg_rotation
            cam_2_global_pose_avg[:3, 3] = avg_translation

            guilder_angle = compute_pitch_angle2(avg_rotation)

            annotated_rgb = draw_frame_axes(
               processed_img, 
               intr, 
               np.zeros(8, dtype=np.float64),
               rvecs, 
               tvecs,
               markers_corners=aruco_detected_corners,
               draw_detected_markers_boxes=False)

            text_lines = [
                f"Depth: {avg_translation[2]:.2f}m",
                f"Angle: {guilder_angle:.2f}deg",
                f"Shift: ({avg_translation[0]:.2f}m, {avg_translation[1]:.2f}m)"
            ]
            y_offset = 40
            midpoint = (annotated_rgb.shape[1] // 2, annotated_rgb.shape[0] // 2)

            annotated_rgb = put_text_lines(
                annotated_rgb,
                text_lines,
                (50, midpoint[1]+y_offset),
                color=(0, 255, 0),
                font_scale=0.8, 
                thickness=2,
                y_offset=y_offset)

            for aruco_corners in aruco_detected_corners:
               aruco_box = aruco_corners[0].astype(np.int16).squeeze()
               box_xyxy = [int(np.min(aruco_box[:, 0])), int(np.min(aruco_box[:, 1])),
                           int(np.max(aruco_box[:, 0])), int(np.max(aruco_box[:, 1]))]

               resized_img = cv2.rectangle(
                   resized_img, 
                   (box_xyxy[0] - offset_px, box_xyxy[1] - offset_px), 
                   (box_xyxy[2] + offset_px, box_xyxy[3] + offset_px), 
                   overlay_color[::-1],
                   thickness=-1)

            guilder_center = cam_2_global_pose_avg[:3, 3]
            guilder_rot_mat = cam_2_global_pose_avg[:3, :3]
            guilder_rot_quat = R.from_matrix(guilder_rot_mat).as_quat(scalar_first=True)

            annot = {
               'camera_intrinsic': intr.tolist(),
               'aruco_detected': aruco_detected,
               'guilder_whl': guilder_whl.tolist(),
               'guilder_center': guilder_center.tolist(),
               'guilder_pitch_deg': guilder_angle,
               'guilder_pitch_rad': math.radians(guilder_angle),
               'guilder_rot': guilder_rot_quat.tolist(),
               'timestamp': img_info['timestamp'],
               'img_width': expected_size[0],
               'img_height': expected_size[1]
            }

            metadata = {'cam2img': intr.tolist()}
            guilder_lhw = guilder_whl.copy()
            guilder_lhw[[0, 1, 2]] = guilder_lhw[[2, 1, 0]]
            rot = [-(math.pi / 2 + math.radians(guilder_angle))]
            box = guilder_center.tolist() + guilder_lhw.tolist() + rot

            pred_instances_3d = InstanceData(metainfo=metadata)
            pred_instances_3d.bboxes_3d = CameraInstance3DBoxes(
                np.array([box], dtype=np.float32), 
                origin=(0.5, 0.5, 0.5))
            pred_instances_3d.labels_3d = np.array([0]) 
            pred_instances_3d.scores_3d = np.array([1.0])

            result = Det3DDataSample(metainfo=metadata)
            result.pred_instances_3d = pred_instances_3d

            annotated_rgb = draw_bboxes(annotated_rgb, result)

            cv2.imwrite(
               os.path.join(annotated_img_dir, img_filename), 
               annotated_rgb)

            cv2.imwrite(
               os.path.join(img_dir, img_filename), 
               resized_img)

            assert img_filename.endswith('.jpg')
            json_path = os.path.join(annotated_dir, img_filename.replace('.jpg', '.json'))

            with open(json_path, 'w') as f:
               json.dump(annot, f, indent=4)

        except Exception as e:
            print(f"Error processing {img_filename}: {e}")
            traceback.print_exc()
            if os.path.exists(img_dir):
                shutil.rmtree(img_dir)
            if os.path.exists(annotated_img_dir):
                shutil.rmtree(annotated_img_dir)
            if os.path.exists(annotated_dir):
                shutil.rmtree(annotated_dir)

100%|██████████| 421/421 [00:12<00:00, 34.72it/s]
